In [12]:
import pandas as pd
# table_train = pd.read_pickle("./table_train_replaced.pkl")
# table_test = pd.read_pickle("./table_test_replaced.pkl")
table_private = pd.read_pickle("./table_private_replaced.pkl")

In [13]:
table_private.head(3)

,ID,text
0,708,小黃薊馬密度高，台南農改場籲加強防治、確保果實品質\n台南區主要水果如檬果及麻豆文旦等正值幼...
1,883,寒流來襲，各類作物應加強寒害防範措施\n依據中央氣象局預報指出，因受強烈大陸冷氣團籠罩，各地...
2,837,水稻秧苗期，慎防水象鼻蟲為害\n行政院農業委員會花蓮區農業改良場表示，目前花蓮地區正值水稻秧...


In [14]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [15]:
import pickle

with open('kw_dict_norm_longest.pickle', 'rb') as handle:
    kw_dict_norm = pickle.load(handle)

kw_dict_add = {"抽穗":2, "蕉株":2}
kw_dict_norm = {**kw_dict_norm, **kw_dict_add}

In [16]:
sentence_list = []
for doc in table_train['text']:
    sentence_list.append(str(doc).replace('\n', ''))
print (len(sentence_list))

420


In [17]:
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER


# ws = WS("./data", disable_cuda=False)
# pos = POS("./data", disable_cuda=False)
# ner = NER("./data", disable_cuda=False)


dictionary = construct_dictionary(kw_dict_norm)
# Run WS-POS pipeline
word_sentence_list = ws(sentence_list, coerce_dictionary=dictionary)
pos_sentence_list = pos(word_sentence_list)

pos_dict = {}
def make_pos_dict(word_sentence, pos_sentence):  # 每一個詞
    assert len(word_sentence) == len(pos_sentence)
    for word, pos in zip(word_sentence, pos_sentence):
        if pos not in pos_dict:
            pos_dict[pos] = set()
        pos_dict[pos].add(word)
    return

for i, sentence in enumerate(sentence_list):  # 每一篇
    make_pos_dict(word_sentence_list[i],  pos_sentence_list[i])



In [20]:
table_private['seg_text'] = word_sentence_list
table_private['seg_text_pos'] = pos_sentence_list

## list the contents of parts of speech

In [18]:
data_items = pos_dict.items()
data_list = list(data_items)
df = pd.DataFrame(data_list)
df

,0,1
0,Na,"{雨害, 高濕度, 禾苗, 太陽, 混合液, 施用量, 根皮, 手錶皮, 靠山, 鐵甲砷酸銨..."
1,VH,"{除外, 優先, 羽化, 如此, 踴躍, 猖厥, 重, 禦寒, 群集, 青蔥, 殆盡, 淺,..."
2,COMMACATEGORY,"{,, ，}"
3,Nc,"{竹崎鄉, 長濱鄉, 園區, 頭城鎮, 監測站, 溪洲鄉, 田畔, 麻豆, 中, 南莊, 研..."
4,VE,"{說, 言, 詢問, 表, 呼籲, 洽詢, 提醒, 嚴防, 傳出, 通報, 強調, 推測, ..."
5,VC,"{開啟, 蛀食, 檢視, 實行, 採取, 擴展, 勘查, 配戴, 俾求, 吹毀, 發布, 毀..."
6,PAUSECATEGORY,{、}
7,A,"{既有, 非法, 輕度, 灰白, 初次, 可用, 下述, 水生, 肩背式, 偶發, 單, 洋..."
8,P,"{直到, 至於, 時值, 因, 距, 用, 順著, 透過, 依據, 藉, 依, 根據, 針對..."
9,Caa,"{～, 至, 以及, 和, 而, 到, 或者, 又, 或, ~, 且, 或是, 乃至, 與, 及}"


In [21]:
table_private

,ID,text,seg_text,seg_text_pos
0,708,小黃薊馬密度高，台南農改場籲加強防治、確保果實品質\n台南區主要水果如檬果及麻豆文旦等正值幼...,"[小黃薊馬, 密度, 高, ，, 台南, 農改場, 籲, 加強, 防治, 、, 確保, 果實...","[Na, Na, VH, COMMACATEGORY, Nc, Nc, VE, VC, VC..."
1,883,寒流來襲，各類作物應加強寒害防範措施\n依據中央氣象局預報指出，因受強烈大陸冷氣團籠罩，各地...,"[寒流, 來襲, ，, 各, 類, 作物, 應, 加強, 寒害, 防範, 措施, 依據, 中...","[Na, VA, COMMACATEGORY, Nes, Nf, Na, D, VC, Na..."
2,837,水稻秧苗期，慎防水象鼻蟲為害\n行政院農業委員會花蓮區農業改良場表示，目前花蓮地區正值水稻秧...,"[水稻, 秧苗期, ，, 慎防, 水, 象鼻蟲, 為害, 行政院, 農業, 委員會, 花蓮區...","[Na, Na, COMMACATEGORY, VC, Na, Na, VC, Nc, Na..."
3,1359,發佈水稻紋枯病及白葉枯病警報\n高屏地區水稻紋枯病發生警報請農友注意防治水稻紋枯病及白葉枯病...,"[發佈, 水稻, 紋枯病, 及, 白葉枯病, 警報, 高屏, 地區, 水稻, 紋枯病, 發生...","[VE, Na, Na, Caa, Na, Na, Nc, Nc, Na, Na, VJ, ..."
4,323,氣候多變溫差大溼度高，台南場籲慎防水稻葉稻熱病\n近日氣溫回升，夜間霧氣籠罩，雲嘉南地區水稻...,"[氣候, 多變, 溫差, 大, 溼度, 高, ，, 台南場, 籲慎, 防, 水稻, 葉稻熱病...","[Na, VH, Na, VH, Na, VH, COMMACATEGORY, Nc, VA..."
...,...,...,...,...
415,316,氣候炎熱小心紅蜘蛛，請農友注意番玉荷包荔枝紅蜘蛛防治。臺東區農改場、防檢局及田邊好幫手關心...,"[氣候, 炎熱, 小心, 紅蜘蛛, ，, 請, 農友, 注意, 番, 玉荷包荔枝, 紅蜘蛛,...","[Na, VH, VK, Na, COMMACATEGORY, VF, Na, VK, Nf..."
416,99,發布本(106)年第一期作水稻葉稻熱病發生預報。桃子園區農改場、防檢局及田邊好幫手關心您。\...,"[發布, 本, (106), 年, 第一, 期, 作, 水稻, 葉稻熱病, 發生, 預報, ...","[VC, Nes, Neu, Nf, Neu, Nf, VC, Na, Na, VJ, Na..."
417,1223,臺灣本土黑螞蟻數量增加，民眾應留意及適時防治。雲林防疫所、防檢局及田邊好幫手關心您。\n近年...,"[臺灣本土黑螞蟻, 數量, 增加, ，, 民眾, 應, 留意, 及, 適時, 防治, 。, ...","[Na, Na, VHC, COMMACATEGORY, Na, D, VK, Caa, D..."
418,1397,水稻陸續抽穗，籲請農友注意防治穗稻熱病、紋枯病及螟蟲。雲林縣政府、防檢局及田邊好幫手關心您。...,"[水稻, 陸續, 抽穗, ，, 籲請, 農友, 注意, 防治, 穗稻熱病, 、, 紋枯病, ...","[Na, D, VA, COMMACATEGORY, VF, Na, VK, VC, Na,..."


In [22]:
# table_train.to_pickle("./table_train_seg.pkl")
# table_test.to_pickle("./table_test_seg.pkl")
table_private.to_pickle("./table_private_seg.pkl")